In [1]:
# leitura dos dados
import pandas as pd
pd.set_option('future.no_silent_downcasting', True)

from pandas.api.types import CategoricalDtype
from sentence_transformers import SentenceTransformer
from tqdm import tqdm

# funções + embeddings
from src.categorizador_batch import categorizar_coluna_batch, preparar_para_categoria

# mapeamentos
from src.categorizar_texto import (
    mapeamento_motivo_fluencia,
    mapeamento_profissoes,
    mapeamento_outros_idiomas,
    mapeamento_problema_aprender,
    mapeamento_escolaridade,
    expandir_mapeamento
)

df_pesquisa_captacao_l12 = pd.read_csv(r"C:\Users\Camilo_Bica\data_science\consultoria\acelerador_petrobras\dados\Pesquisa_SSP.csv")
df_compradores = pd.read_csv(r"C:\Users\Camilo_Bica\data_science\consultoria\acelerador_petrobras\dados\Compradores _ SSP-L12.csv")
df_leads_trafego = pd.read_csv(r"C:\Users\Camilo_Bica\data_science\consultoria\acelerador_petrobras\dados\Dados Leads _ SSP_trafego.csv")
df_leads_invest_trafego = pd.read_csv(r"C:\Users\Camilo_Bica\data_science\consultoria\acelerador_petrobras\dados\Dados Leads _ SSP_inves_trafego.csv")

In [2]:
df_pesquisa_captacao_l12 = df_pesquisa_captacao_l12.drop(["Funil"], axis=1, errors='ignore')
df_pesquisa_captacao_l12.rename(columns={'E-mail': 'email'}, inplace=True)
df_pesquisa_captacao_l12.columns = df_pesquisa_captacao_l12.columns.str.lower()

df_pesquisa_captacao_l12['data'] = pd.to_datetime(df_pesquisa_captacao_l12['data'], errors='coerce')
df_pesquisa_captacao_l12['data'] = df_pesquisa_captacao_l12['data'].dt.tz_localize(None)
df_pesquisa_captacao_l12['data'] = df_pesquisa_captacao_l12['data'].dt.normalize()

df_pesquisa_captacao_l12["email"] = df_pesquisa_captacao_l12["email"].str.lower().str.strip()
df_pesquisa_captacao_l12 = df_pesquisa_captacao_l12.drop_duplicates(subset='email', keep='first')

df_pesquisa_captacao_l12.info()

<class 'pandas.core.frame.DataFrame'>
Index: 26201 entries, 0 to 26934
Data columns (total 9 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   data                   26201 non-null  datetime64[ns]
 1   email                  26201 non-null  object        
 2   idade                  26201 non-null  object        
 3   nível                  26201 non-null  object        
 4   situação profissional  26201 non-null  object        
 5   renda                  26201 non-null  object        
 6   escolaridade           26201 non-null  object        
 7   duvida                 26200 non-null  object        
 8   dificuldade            26200 non-null  object        
dtypes: datetime64[ns](1), object(8)
memory usage: 2.0+ MB


In [3]:
df_pesquisa_captacao_l12["renda"] = (
    df_pesquisa_captacao_l12["renda"]
    .replace({
        "O a R$1.000\u2028": "De 0 a 1.000",
        "R$1.000 a R$3.000": "De 1.000 a 3.000",
        "R$3.000 a R$5.000\u2028": "De 3.000 a 5.000",
        "R$5.000 a R$10.000\u2028": "De 5.000 a 10.000",
        "Mais de R$10.000": "Mais de 10.000"
    })
)

df_pesquisa_captacao_l12["escolaridade"] = (
    df_pesquisa_captacao_l12["escolaridade"]
    .replace({
        "Ensino superior incompleto\u2028": "Ensino superior incompleto"
    })
)

In [4]:
# Filtrar o DataFrame para excluir linhas onde o email contém "teste" ou "x@x"
df_pesquisa_captacao_l12 = df_pesquisa_captacao_l12[
    ~df_pesquisa_captacao_l12['email'].str.contains('teste|x@x', case=False, na=False)
]

df_pesquisa_captacao_l12

,data,email,idade,nível,situação profissional,renda,escolaridade,duvida,dificuldade
2,2025-04-07,manoeltarsis22@gmail.com,33-37,Técnica,Autônomo,De 1.000 a 3.000,Ensino médio completo,O que preciso pra ser aprovado,Tempo
3,2025-04-07,geovanarodrigues0106@gmail.com,23-27,Técnica,Desempregado(a),Estou desempregado(a),Ensino superior incompleto,.,Separa material
4,2025-04-07,jj272824@gmail.com,18-22,Superior,Desempregado(a),Estou desempregado(a),Ensino médio completo,O quão difícil é conseguir passar,Nenhuma
5,2025-04-07,iuriguerreiro45@gmail.com,28-32,Técnica,Desempregado(a),Estou desempregado(a),Nível técnico completo,Em qual lugar do Brasil pode fazer o concurso?,Tempo
6,2025-04-07,stevaogabrieli2020@gmail.com,18-22,Técnica,Autônomo,Estou desempregado(a),Ensino médio completo,Se precisa de ensino superior,Nenhuma
...,...,...,...,...,...,...,...,...,...
26930,2025-04-24,micaellgmarcelini@gmail.com,23-27,Técnica,Desempregado(a),De 1.000 a 3.000,Nível técnico completo,O que vai cair,Tempo e disposição
26931,2025-04-24,samuelsljsantos@gmail.com,23-27,Técnica,Funcionário público,De 1.000 a 3.000,Ensino médio completo,Aonde se inscrever,Depois do trabalho
26932,2025-04-24,rleao4258@gmail.com,18-22,Superior,Autônomo,De 0 a 1.000,Ensino médio completo,O perigo,tem um roteiro de estudo
26933,2025-04-25,duartetomadocci@yahoo.com.br,46-52,Técnica,Autônomo,De 3.000 a 5.000,Ensino médio completo,Grau de dificuldade,Tempo


In [5]:
df_leads_trafego.rename(columns={'E-mail': 'email'}, inplace=True)
df_leads_trafego.columns = df_leads_trafego.columns.str.lower()

df_leads_trafego['data'] = pd.to_datetime(df_leads_trafego['data'], errors='coerce')
df_leads_trafego['data'] = df_leads_trafego['data'].dt.tz_localize(None)
df_leads_trafego['data'] = df_leads_trafego['data'].dt.normalize()

df_leads_trafego["email"] = df_leads_trafego["email"].str.lower().str.strip()
df_leads_trafego = df_leads_trafego.drop_duplicates(subset='email', keep='first')

df_leads_trafego.info()

<class 'pandas.core.frame.DataFrame'>
Index: 43213 entries, 0 to 43278
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   data          43213 non-null  datetime64[ns]
 1   nome          43213 non-null  object        
 2   email         43213 non-null  object        
 3   utm source    41846 non-null  object        
 4   utm campaign  31841 non-null  object        
 5   utm medium    41843 non-null  object        
 6   utm content   31841 non-null  object        
 7   tag           15534 non-null  object        
dtypes: datetime64[ns](1), object(7)
memory usage: 3.0+ MB


C:\Users\Camilo_Bica\AppData\Local\Temp\ipykernel_13812\2763487327.py:4: UserWarning: Parsing dates in %d/%m/%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  df_leads_trafego['data'] = pd.to_datetime(df_leads_trafego['data'], errors='coerce')


In [6]:
# Filtrar o DataFrame para excluir linhas onde o email contém "teste" ou "x@x"
df_leads_trafego = df_leads_trafego[
    ~df_leads_trafego['utm source'].str.contains('teste|NaN', case=False, na=False)
]

# Definir as colunas UTM que devem ser completas
colunas_utms = ['utm source', 'utm campaign', 'utm medium', 'utm content']

# Apenas remover se todas as UTMs forem NaN
df_leads_trafego = df_leads_trafego[~df_leads_trafego[colunas_utms].isna().all(axis=1)].copy()

# Resetar índice
df_leads_trafego = df_leads_trafego.reset_index(drop=True)

df_leads_trafego

,data,nome,email,utm source,utm campaign,utm medium,utm content,tag
0,2025-03-31,Hermes,hermesc1@gmail.com,Facebook-Ads,SSP-L12 | CAPTACAO | CBO | FRIO | ALL | BR | 3...,00 | AUTO | LAL | 1% | ENVOLVIMENTO IG 7D,ADS_002_ANUNCIO PETRO 02,SSP-L12
1,2025-03-31,Geisa,santos.geisa.lucas@gmail.com,Facebook-Ads,SSP-L12 | CAPTACAO | CBO | FRIO | ALL | BR | 3...,00 | AUTO | LAL | 1% | ENVOLVIMENTO IG 7D,ADS_002_ANUNCIO PETRO 02,SSP-L12
2,2025-03-31,Uelington,uelingtonreis25@gmail.com,Facebook-Ads,SSP-L12 | CAPTACAO | CBO | FRIO | ALL | BR | 3...,00 | AUTO | LAL | 1% | ENVOLVIMENTO IG 14D,ADS_003_ANUNCIO PETRO 03,SSP-L12
3,2025-03-31,Vivian,vivian.serafim.xoxo@gmail.com,Facebook-Ads,SSP-L12 | CAPTACAO | CBO | FRIO | ALL | BR | 3...,00 | AUTO | LAL | 1% | ENVOLVIMENTO IG 14D,ADS_003_ANUNCIO PETRO 03,SSP-L12
4,2025-03-31,leonardo,leonardolopesviana@gmail.com,Facebook-Ads,SSP-L12 | CAPTACAO | CBO | FRIO | ALL | BR | 3...,00 | AUTO | LAL | 1% | ENVOLVIMENTO IG 7D,ADS_001_ANUNCIO PETRO 01,SSP-L12
...,...,...,...,...,...,...,...,...
41840,2025-04-17,Luis,lh5778006@gmail.com,Youtube,NaN,Descricao,NaN,NaN
41841,2025-04-17,KEVEN,kevenazevedomercado@gmail.com,Youtube,NaN,Descricao,NaN,NaN
41842,2025-04-23,Danilo,danilofiais155@gmail.com,Facebook-Ads,{{campaign.name}},{{adset.name}},{{ad.name}},NaN
41843,2025-04-23,Cristiano,lendasocapa9@gmail.com,Facebook-Ads,{{campaign.name}},{{adset.name}},{{ad.name}},NaN


In [7]:
df_leads_invest_trafego['date'] = pd.to_datetime(df_leads_invest_trafego['date'], errors='coerce')
df_leads_invest_trafego['date'] = df_leads_invest_trafego['date'].dt.tz_localize(None)
df_leads_invest_trafego['date'] = df_leads_invest_trafego['date'].dt.normalize()

df_leads_invest_trafego.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 788 entries, 0 to 787
Data columns (total 8 columns):
 #   Column                     Non-Null Count  Dtype         
---  ------                     --------------  -----         
 0   date                       788 non-null    datetime64[ns]
 1   campaign                   788 non-null    object        
 2   adset_name                 788 non-null    object        
 3   ad_name                    788 non-null    object        
 4   spend                      788 non-null    object        
 5   impressions                788 non-null    int64         
 6   actions_link_click         788 non-null    int64         
 7   actions_landing_page_view  788 non-null    int64         
dtypes: datetime64[ns](1), int64(3), object(4)
memory usage: 49.4+ KB


C:\Users\Camilo_Bica\AppData\Local\Temp\ipykernel_13812\352342578.py:1: UserWarning: Parsing dates in %d/%m/%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  df_leads_invest_trafego['date'] = pd.to_datetime(df_leads_invest_trafego['date'], errors='coerce')


In [8]:
# Lista das colunas que queremos validar
colunas_investimento = ['spend', 'impressions', 'actions_link_click', 'actions_landing_page_view']

# Garantir que essas colunas sejam numéricas (importante!)
for col in colunas_investimento:
    df_leads_invest_trafego[col] = (
        df_leads_invest_trafego[col]
        .astype(str)
        .str.replace(',', '.', regex=False)
        .astype(float)
    )

# Agora, remover linhas onde TODAS essas colunas são zero
df_leads_invest_trafego = df_leads_invest_trafego[
    ~(df_leads_invest_trafego[colunas_investimento] == 0).all(axis=1)
].copy()

# Ajustar df_leads_invest_trafego para nomes iguais
df_leads_invest_trafego = df_leads_invest_trafego.rename(columns={
    'date': 'data',
    'campaign': 'utm campaign',
    'adset_name': 'utm medium',
    'ad_name': 'utm content'
})

# Resetar índice (opcional, para organização)
df_leads_invest_trafego = df_leads_invest_trafego.reset_index(drop=True)

In [9]:
df_leads_invest_trafego

,data,utm campaign,utm medium,utm content,spend,impressions,actions_link_click,actions_landing_page_view
0,2025-03-31,SSP-L12 | CAPTACAO | CBO | FRIO | ALL | BR | 3...,00 | AUTO | LAL | 1% | ENVOLVIMENTO IG 7D,ADS_003_ANUNCIO PETRO 03,40.04,2878.0,16.0,1.0
1,2025-03-31,SSP-L12 | CAPTACAO | CBO | FRIO | ALL | BR | 3...,00 | AUTO | LAL | 1% | ENVOLVIMENTO IG 7D,ADS_002_ANUNCIO PETRO 02,86.64,5862.0,41.0,1.0
2,2025-03-31,SSP-L12 | CAPTACAO | CBO | FRIO | ALL | BR | 3...,00 | AUTO | LAL | 1% | ENVOLVIMENTO IG 7D,ADS_001_ANUNCIO PETRO 01,204.66,17136.0,259.0,20.0
3,2025-03-31,SSP-L12 | CAPTACAO | CBO | FRIO | ALL | BR | 3...,00 | AUTO | LAL | 1% | ENVOLVIMENTO IG 14D,ADS_003_ANUNCIO PETRO 03,78.85,6254.0,30.0,0.0
4,2025-03-31,SSP-L12 | CAPTACAO | CBO | FRIO | ALL | BR | 3...,00 | AUTO | LAL | 1% | ENVOLVIMENTO IG 14D,ADS_002_ANUNCIO PETRO 02,20.32,1786.0,14.0,1.0
...,...,...,...,...,...,...,...,...
624,2025-04-15,SSP-L12 | CAPTACAO | CBO | QUENTE | ALL | BR |...,00 | AUTO | SALVOU + ENVIOU MSG | 365D,ADS_001_ANUNCIO PETRO 01,0.04,5.0,0.0,0.0
625,2025-04-15,SSP-L12 | CAPTACAO | CBO | QUENTE | ALL | BR |...,00 | AUTO | ENVOLVIMENTO | 90D,ADS_003_ANUNCIO PETRO 03,8.54,670.0,9.0,7.0
626,2025-04-15,SSP-L12 | CAPTACAO | CBO | QUENTE | ALL | BR |...,00 | AUTO | ENVOLVIMENTO | 90D,ADS_001_ANUNCIO PETRO 01,13.27,925.0,13.0,8.0
627,2025-04-15,SSP-L12 | CAPTACAO | CBO | QUENTE | ALL | BR |...,00 | AUTO | ENVOLVIMENTO | 14D,ADS_001_ANUNCIO PETRO 01,16.90,1043.0,9.0,8.0


In [10]:
df_compradores.rename(columns={'email contato': 'email'}, inplace=True)
df_compradores["email"] = df_compradores["email"].str.lower().str.strip()
df_compradores = df_compradores.drop_duplicates(subset='email', keep='first')

df_compradores.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 593 entries, 0 to 592
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   email   593 non-null    object
dtypes: object(1)
memory usage: 4.8+ KB


In [11]:
df_pesquisa_compradores_l12 = df_pesquisa_captacao_l12[
    df_pesquisa_captacao_l12["email"].isin(df_compradores["email"])
].copy()

In [12]:
df_pesquisa_compradores_l12.info()

<class 'pandas.core.frame.DataFrame'>
Index: 194 entries, 12 to 26875
Data columns (total 9 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   data                   194 non-null    datetime64[ns]
 1   email                  194 non-null    object        
 2   idade                  194 non-null    object        
 3   nível                  194 non-null    object        
 4   situação profissional  194 non-null    object        
 5   renda                  194 non-null    object        
 6   escolaridade           194 non-null    object        
 7   duvida                 194 non-null    object        
 8   dificuldade            194 non-null    object        
dtypes: datetime64[ns](1), object(8)
memory usage: 15.2+ KB


In [13]:
df_pesquisa_compradores_l12

,data,email,idade,nível,situação profissional,renda,escolaridade,duvida,dificuldade
12,2025-04-07,caroline.micaelle16@gmail.com,18-22,Superior,Funcionário de empresa privada,De 1.000 a 3.000,Ensino superior completo,A quais vagas posso concorrer tendo tecnólogo ...,Tempo
60,2025-04-07,helder170@yahoo.com.br,38-45,Técnica,Funcionário de empresa privada,De 1.000 a 3.000,Ensino médio completo,Tenho o curso técnico em contabilidade servi p...,Memorização
270,2025-04-07,joaochodachi@gmail.com,28-32,Técnica,Funcionário de empresa privada,De 1.000 a 3.000,Ensino médio completo,Em qual cidade irei fazer o concurso?,Não sei o que estudar
295,2025-04-07,mayconjean612@gmail.com,28-32,Técnica,Desempregado(a),De 1.000 a 3.000,Ensino superior completo,Quando vai sair o edital,Foco
309,2025-04-07,cidgmj79@gmail.com,46-52,Técnica,Funcionário de empresa privada,De 3.000 a 5.000,Nível técnico completo,A prova,Tempo
...,...,...,...,...,...,...,...,...,...
26635,2025-04-15,isabela.a.rezende@gmail.com,33-37,Técnica,Funcionário de empresa privada,De 3.000 a 5.000,Ensino superior completo,A minha dúvida principal é de como é o trabalh...,"Concentração, porque tem muito tempo q não par..."
26689,2025-04-16,espindolavitorandrade@gmail.com,23-27,Superior,Desempregado(a),De 0 a 1.000,Ensino médio completo,Consigo fazer sem ter um técnico,Tempo
26726,2025-04-16,felipebarboosa532@gmail.com,18-22,Técnica,Funcionário de empresa privada,De 1.000 a 3.000,Ensino médio completo,Matérias,Nenhuma
26805,2025-04-18,davisouzamg1@gmail.com,23-27,Técnica,Funcionário de empresa privada,De 1.000 a 3.000,Nível técnico completo,Formação,Ensino


In [14]:
df_compradores_nao_respondentes = df_compradores[
    ~df_compradores["email"].isin(df_pesquisa_captacao_l12["email"])
].copy()

In [15]:
df_compradores_nao_respondentes.info()

<class 'pandas.core.frame.DataFrame'>
Index: 399 entries, 0 to 591
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   email   399 non-null    object
dtypes: object(1)
memory usage: 6.2+ KB


In [16]:
df_pesquisa_captacao_l12['lancamentos'] = 'SSP-L12'
df_pesquisa_compradores_l12['lancamentos'] = 'SSP-L12'

In [17]:
# Colunas de UTM que vamos trazer
colunas_utm = ['utm source', 'utm campaign', 'utm medium', 'utm content']

# Fazer o merge trazendo as UTMs para df_pesquisa_captacao_l12
df_pesquisa_captacao_l12 = df_pesquisa_captacao_l12.merge(
    df_leads_trafego[['email'] + colunas_utm],
    on='email',
    how='left'  # Mantém todas as respostas da pesquisa, mesmo se não tiver UTM
)

# Fazer o merge trazendo as UTMs para df_pesquisa_compradores_l12
df_pesquisa_compradores_l12 = df_pesquisa_compradores_l12.merge(
    df_leads_trafego[['email'] + colunas_utm],
    on='email',
    how='left'
)

In [18]:
df_pesquisa_captacao_l12

,data,email,idade,nível,situação profissional,renda,escolaridade,duvida,dificuldade,lancamentos,utm source,utm campaign,utm medium,utm content
0,2025-04-07,manoeltarsis22@gmail.com,33-37,Técnica,Autônomo,De 1.000 a 3.000,Ensino médio completo,O que preciso pra ser aprovado,Tempo,SSP-L12,Facebook-Ads,SSP-L12 | CAPTACAO | CBO | QUENTE | ALL | BR |...,00 | AUTO | MIX DE PUBLICOS | CRIATIVO 07,ADS_007_ANUNCIO PETRO 07
1,2025-04-07,geovanarodrigues0106@gmail.com,23-27,Técnica,Desempregado(a),Estou desempregado(a),Ensino superior incompleto,.,Separa material,SSP-L12,Facebook-Ads,SSP-L12 | CAPTACAO | CBO | FRIO | ALL | BR | 3...,00 | AUTO | LAL | 1% | ENVOLVIMENTO IG 7D,ADS_001_ANUNCIO PETRO 01
2,2025-04-07,jj272824@gmail.com,18-22,Superior,Desempregado(a),Estou desempregado(a),Ensino médio completo,O quão difícil é conseguir passar,Nenhuma,SSP-L12,Facebook-Ads,SSP-L12 | CAPTACAO | CBO | QUENTE | ALL | BR |...,00 | AUTO | ENVOLVIMENTO | 90D,ADS_006_ANUNCIO PETRO 06
3,2025-04-07,iuriguerreiro45@gmail.com,28-32,Técnica,Desempregado(a),Estou desempregado(a),Nível técnico completo,Em qual lugar do Brasil pode fazer o concurso?,Tempo,SSP-L12,Facebook-Ads,SSP-L12 | CAPTACAO | CBO | QUENTE | ALL | BR |...,00 | AUTO | ENVOLVIMENTO | 30D,ADS_001_ANUNCIO PETRO 01
4,2025-04-07,stevaogabrieli2020@gmail.com,18-22,Técnica,Autônomo,Estou desempregado(a),Ensino médio completo,Se precisa de ensino superior,Nenhuma,SSP-L12,Facebook-Ads,SSP-L12 | CAPTACAO | CBO | QUENTE | ALL | BR |...,00 | AUTO | MIX DE PUBLICOS | CRIATIVO 08,ADS_008_ANUNCIO PETRO 08
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26190,2025-04-24,micaellgmarcelini@gmail.com,23-27,Técnica,Desempregado(a),De 1.000 a 3.000,Nível técnico completo,O que vai cair,Tempo e disposição,SSP-L12,NaN,NaN,NaN,NaN
26191,2025-04-24,samuelsljsantos@gmail.com,23-27,Técnica,Funcionário público,De 1.000 a 3.000,Ensino médio completo,Aonde se inscrever,Depois do trabalho,SSP-L12,NaN,NaN,NaN,NaN
26192,2025-04-24,rleao4258@gmail.com,18-22,Superior,Autônomo,De 0 a 1.000,Ensino médio completo,O perigo,tem um roteiro de estudo,SSP-L12,Facebook-Ads,{{campaign.name}},{{adset.name}},{{ad.name}}
26193,2025-04-25,duartetomadocci@yahoo.com.br,46-52,Técnica,Autônomo,De 3.000 a 5.000,Ensino médio completo,Grau de dificuldade,Tempo,SSP-L12,NaN,NaN,NaN,NaN


In [19]:
df_pesquisa_compradores_l12

,data,email,idade,nível,situação profissional,renda,escolaridade,duvida,dificuldade,lancamentos,utm source,utm campaign,utm medium,utm content
0,2025-04-07,caroline.micaelle16@gmail.com,18-22,Superior,Funcionário de empresa privada,De 1.000 a 3.000,Ensino superior completo,A quais vagas posso concorrer tendo tecnólogo ...,Tempo,SSP-L12,Youtube,NaN,Descricao,NaN
1,2025-04-07,helder170@yahoo.com.br,38-45,Técnica,Funcionário de empresa privada,De 1.000 a 3.000,Ensino médio completo,Tenho o curso técnico em contabilidade servi p...,Memorização,SSP-L12,Facebook-Ads,SSP-L12 | CAPTACAO | ABO | FRIO | ALL | BR | 0...,00 | AUTO | LAL | 1% | ENV 7D | CRIATIVO 10,ADS_010_ANUNCIO PETRO 10
2,2025-04-07,joaochodachi@gmail.com,28-32,Técnica,Funcionário de empresa privada,De 1.000 a 3.000,Ensino médio completo,Em qual cidade irei fazer o concurso?,Não sei o que estudar,SSP-L12,Youtube,NaN,Descricao,NaN
3,2025-04-07,mayconjean612@gmail.com,28-32,Técnica,Desempregado(a),De 1.000 a 3.000,Ensino superior completo,Quando vai sair o edital,Foco,SSP-L12,Facebook-Ads,SSP-L12 | CAPTACAO | CBO | QUENTE | ALL | BR |...,00 | AUTO | ENVOLVIMENTO | 90D,ADS_007_ANUNCIO PETRO 07
4,2025-04-07,cidgmj79@gmail.com,46-52,Técnica,Funcionário de empresa privada,De 3.000 a 5.000,Nível técnico completo,A prova,Tempo,SSP-L12,Facebook-Ads,SSP-L12 | CAPTACAO | CBO | FRIO | ALL | BR | 3...,00 | AUTO | LAL | 1% | ENVOLVIMENTO IG 14D,ADS_001_ANUNCIO PETRO 01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
189,2025-04-15,isabela.a.rezende@gmail.com,33-37,Técnica,Funcionário de empresa privada,De 3.000 a 5.000,Ensino superior completo,A minha dúvida principal é de como é o trabalh...,"Concentração, porque tem muito tempo q não par...",SSP-L12,Youtube,NaN,Descricao,NaN
190,2025-04-16,espindolavitorandrade@gmail.com,23-27,Superior,Desempregado(a),De 0 a 1.000,Ensino médio completo,Consigo fazer sem ter um técnico,Tempo,SSP-L12,NaN,NaN,NaN,NaN
191,2025-04-16,felipebarboosa532@gmail.com,18-22,Técnica,Funcionário de empresa privada,De 1.000 a 3.000,Ensino médio completo,Matérias,Nenhuma,SSP-L12,Youtube,NaN,Descricao,NaN
192,2025-04-18,davisouzamg1@gmail.com,23-27,Técnica,Funcionário de empresa privada,De 1.000 a 3.000,Nível técnico completo,Formação,Ensino,SSP-L12,Facebook-Ads,SSP-L12 | CAPTACAO | CBO | FRIO | ALL | BR | 3...,00 | AUTO | LAL | 1% | ENVOLVIMENTO IG 14D,ADS_007_ANUNCIO PETRO 07


In [30]:
# Definir as colunas UTM
colunas_utms = ['utm source', 'utm campaign', 'utm medium', 'utm content']

# Contar quantas linhas têm todas as UTMs como NaN
qtd_todas_utms_nan = df_pesquisa_compradores_l12[colunas_utms].isna().all(axis=1).sum()

print(f"Quantidade de linhas com TODAS as UTMs NaN: {qtd_todas_utms_nan}")

Quantidade de linhas com TODAS as UTMs NaN: 20


In [20]:
# Definir colunas categóricas não ordenadas
colunas_categoricas_nao_ordenadas = [
    "email",
    "situação profissional",
    "duvida",
    "dificuldade",
    "campaign",
    "adset_name",
    "ad_name",
    "nome",
    "utm source",
    "utm campaign",
    "utm medium",
    "utm countet",
    "tag"
]

# Definir colunas categóricas ordenadas 
colunas_categoricas_ordenadas = [
    "idade",
    "nível",
    "renda",
    "escolaridade",
    "lancamentos"
]

# Definir colunas de datas ordenadas
colunas_datas = [
    "data",
    "date"
]

# Função para identificar colunas numéricas em qualquer DataFrame
def identificar_colunas_numericas(df, cat_nao_ord, cat_ord, datas):
    return [
        coluna for coluna in df.columns
        if coluna not in (cat_nao_ord + cat_ord + datas)
        and pd.api.types.is_numeric_dtype(df[coluna])
    ]

# Aplicar a função nos DataFrames
colunas_numericas_pesquisa = identificar_colunas_numericas(df_pesquisa_captacao_l12, colunas_categoricas_nao_ordenadas, colunas_categoricas_ordenadas, colunas_datas)
colunas_numericas_compradores = identificar_colunas_numericas(df_compradores, colunas_categoricas_nao_ordenadas, colunas_categoricas_ordenadas, colunas_datas)
colunas_numericas_leads_trafego = identificar_colunas_numericas(df_leads_trafego, colunas_categoricas_nao_ordenadas, colunas_categoricas_ordenadas, colunas_datas)
colunas_numericas_leads_invest_trafego = identificar_colunas_numericas(df_leads_invest_trafego, colunas_categoricas_nao_ordenadas, colunas_categoricas_ordenadas, colunas_datas)

In [21]:
# Cópia explícita dos DataFrames para evitar SettingWithCopyWarning
df_pesquisa_captacao_l12 = df_pesquisa_captacao_l12.copy()

# Mapeamento de colunas ordenadas com suas categorias
categorias_ordenadas = {
    'idade': ['18-22', '23-27', '28-32', '33-37', '38-45', '46-52', '53 ou mais'],
    'escolaridade': [
        'Ensino médio completo', 'Nível técnico incompleto', 'Nível técnico completo',
        'Ensino superior incompleto', 'Ensino superior completo'
    ],
    'renda': [
        'Estou desempregado(a)', 'De 0 a 1.000', 'De 1.000 a 3.000', 'De 3.000 a 5.000',
        'De 5.000 a 10.000', 'Mais de 10.000'
    ],
    'nível': [
        'Técnica', 'Superior'
    ]
}

# Função para aplicar a ordenação de categorias
def aplicar_categorias_ordenadas(df, categorias):
    for coluna, ordem in categorias.items():
        if coluna in df.columns:
            df[coluna] = df[coluna].astype("category").cat.set_categories(ordem, ordered=True)
    return df

# Aplicar nos dois DataFrames
df_pesquisa_captacao_l12 = aplicar_categorias_ordenadas(df_pesquisa_captacao_l12, categorias_ordenadas)

In [22]:
# Função para aplicar conversão para "category" não ordenado
def aplicar_categorias_nao_ordenadas(df, colunas):
    for col in colunas:
        if col in df.columns:
            df[col] = df[col].astype("category")
    return df

# Aplicar nos dois DataFrames
df_pesquisa_captacao_l12 = aplicar_categorias_nao_ordenadas(df_pesquisa_captacao_l12.copy(), colunas_categoricas_nao_ordenadas)
df_leads_trafego = aplicar_categorias_nao_ordenadas(df_leads_trafego.copy(), colunas_categoricas_nao_ordenadas)
df_leads_invest_trafego = aplicar_categorias_nao_ordenadas(df_leads_invest_trafego.copy(), colunas_categoricas_nao_ordenadas)
df_compradores = aplicar_categorias_nao_ordenadas(df_compradores.copy(), colunas_categoricas_nao_ordenadas)

In [23]:
def explorar_valores_unicos(df, colunas, nome_df="DataFrame"):
    print(f"\n--- Valores únicos em {nome_df} ---")
    for coluna in colunas:
        if coluna in df.columns:
            print(f"{coluna}: {df[coluna].unique()}")
            print()
        else:
            print(f"{coluna}: [coluna não encontrada em {nome_df}]\n")

# Unir todas as colunas categóricas
todas_categoricas = colunas_categoricas_nao_ordenadas + colunas_categoricas_ordenadas

# Explorar os dois DataFrames
explorar_valores_unicos(df_pesquisa_captacao_l12, todas_categoricas, nome_df="df_pesquisa_captacao_l12")
explorar_valores_unicos(df_leads_trafego, todas_categoricas, nome_df="df_leads_trafego")
explorar_valores_unicos(df_leads_invest_trafego, todas_categoricas, nome_df="df_leads_invest_trafego")
explorar_valores_unicos(df_compradores, todas_categoricas, nome_df="df_compradores")


--- Valores únicos em df_pesquisa_captacao_l12 ---
email: ['manoeltarsis22@gmail.com', 'geovanarodrigues0106@gmail.com', 'jj272824@gmail.com', 'iuriguerreiro45@gmail.com', 'stevaogabrieli2020@gmail.com', ..., 'micaellgmarcelini@gmail.com', 'samuelsljsantos@gmail.com', 'rleao4258@gmail.com', 'duartetomadocci@yahoo.com.br', 'luciapereirabnps@gmail.com']
Length: 26195
Categories (26195, object): ['007filipemonteiro@gmail.com', '015988177447c@gmail.com', '015antonyrl@gmail.com', '01charles18@gmail.com', ..., 'zydannesantos701@gmail.com', 'zyellbr@gmail.com', 'zzbcxhh@gmail.com', 'zzgundam83@hotmail.com']

situação profissional: ['Autônomo', 'Desempregado(a)', 'Funcionário de empresa privada', 'Empresário', 'Funcionário público', 'Já sou funcionário terceirizado do sistema Pe...]
Categories (6, object): ['Autônomo', 'Desempregado(a)', 'Empresário', 'Funcionário de empresa privada', 'Funcionário público', 'Já sou funcionário terceirizado do sistema Pe...]

duvida: ['O que preciso pra ser ap

In [24]:
"""
model = SentenceTransformer("all-MiniLM-L6-v2")

# Mapas distintos por coluna
mapeamentos = {
    'problema_aprender': mapeamento_problema_aprender,
    'profissao': mapeamento_profissoes,
    'fala_outro_idioma': mapeamento_outros_idiomas,
    'motivo_fluencia_espanhol': mapeamento_motivo_fluencia,
    'escolaridade': mapeamento_escolaridade
}

for coluna, mapeamento in mapeamentos.items():
    nome_coluna_categoria = f"{coluna}_categoria"

    # Expandir mapeamento para esta coluna
    mapeamento_expandido, mapeamento_embeddings = preparar_para_categoria(mapeamento, model)

    # Categorização com o mapeamento certo
    df_pesquisa_captacao_lancamentos.loc[:, nome_coluna_categoria] = categorizar_coluna_batch(
        df_pesquisa_captacao_lancamentos[coluna],
        mapeamento_expandido,
        mapeamento_embeddings,
        model,
        threshold=0.6,
        desc=coluna
    )
"""

'\nmodel = SentenceTransformer("all-MiniLM-L6-v2")\n\n# Mapas distintos por coluna\nmapeamentos = {\n    \'problema_aprender\': mapeamento_problema_aprender,\n    \'profissao\': mapeamento_profissoes,\n    \'fala_outro_idioma\': mapeamento_outros_idiomas,\n    \'motivo_fluencia_espanhol\': mapeamento_motivo_fluencia,\n    \'escolaridade\': mapeamento_escolaridade\n}\n\nfor coluna, mapeamento in mapeamentos.items():\n    nome_coluna_categoria = f"{coluna}_categoria"\n\n    # Expandir mapeamento para esta coluna\n    mapeamento_expandido, mapeamento_embeddings = preparar_para_categoria(mapeamento, model)\n\n    # Categorização com o mapeamento certo\n    df_pesquisa_captacao_lancamentos.loc[:, nome_coluna_categoria] = categorizar_coluna_batch(\n        df_pesquisa_captacao_lancamentos[coluna],\n        mapeamento_expandido,\n        mapeamento_embeddings,\n        model,\n        threshold=0.6,\n        desc=coluna\n    )\n'

In [25]:
"""# Exibir strings completas
pd.set_option("display.max_colwidth", None)

# Colunas categorizadas e respectivas colunas originais
categorias = {
    "problema_aprender": "problema_aprender_categoria",
    "profissao": "profissao_categoria",
    "fala_outro_idioma": "fala_outro_idioma_categoria",
    "motivo_fluencia_espanhol": "motivo_fluencia_espanhol_categoria",
    "escolaridade": "escolaridade_categoria"
}

# Categorias problemáticas
categorias_problema = ["Outros"]

# Loop para filtrar e exibir como DataFrame formatado
for original_col, categoria_col in categorias.items():
    for problema in categorias_problema:
        print(f"\n=== Valores de '{original_col}' categorizados como '{problema}' ===")

        filtro = df_pesquisa_captacao_lancamentos[categoria_col] == problema
        valores = df_pesquisa_captacao_lancamentos.loc[filtro, original_col].value_counts()

        # Envolver cada valor com aspas
        valores.index = [f'"{val}"' for val in valores.index]

        # Converter para DataFrame para visualização completa
        valores_df = valores.reset_index().head(300)
        valores_df.columns = ['valor_original', 'frequencia']

        # Exibir o resultado completo
        print(valores_df.to_string(index=False))
"""

'# Exibir strings completas\npd.set_option("display.max_colwidth", None)\n\n# Colunas categorizadas e respectivas colunas originais\ncategorias = {\n    "problema_aprender": "problema_aprender_categoria",\n    "profissao": "profissao_categoria",\n    "fala_outro_idioma": "fala_outro_idioma_categoria",\n    "motivo_fluencia_espanhol": "motivo_fluencia_espanhol_categoria",\n    "escolaridade": "escolaridade_categoria"\n}\n\n# Categorias problemáticas\ncategorias_problema = ["Outros"]\n\n# Loop para filtrar e exibir como DataFrame formatado\nfor original_col, categoria_col in categorias.items():\n    for problema in categorias_problema:\n        print(f"\n=== Valores de \'{original_col}\' categorizados como \'{problema}\' ===")\n\n        filtro = df_pesquisa_captacao_lancamentos[categoria_col] == problema\n        valores = df_pesquisa_captacao_lancamentos.loc[filtro, original_col].value_counts()\n\n        # Envolver cada valor com aspas\n        valores.index = [f\'"{val}"\' for val in

In [26]:
"""df_pesquisa_captacao_lancamentos = df_pesquisa_captacao_lancamentos.drop([
    'profissao',
    'fala_outro_idioma',
    'motivo_fluencia_espanhol',
    'escolaridade',
    'problema_aprender'
], axis=1, errors='ignore')
"""

"df_pesquisa_captacao_lancamentos = df_pesquisa_captacao_lancamentos.drop([\n    'profissao',\n    'fala_outro_idioma',\n    'motivo_fluencia_espanhol',\n    'escolaridade',\n    'problema_aprender'\n], axis=1, errors='ignore')\n"

In [27]:
caminho_csv_lead = r"C:\Users\Camilo_Bica\data_science\consultoria\acelerador_petrobras\dados\pesquisa_captacao.csv"
df_pesquisa_captacao_l12.to_csv(caminho_csv_lead, index=False, encoding='utf-8-sig')

caminho_csv_aluno = r"C:\Users\Camilo_Bica\data_science\consultoria\acelerador_petrobras\dados\pesquisa_aluno.csv"
df_pesquisa_compradores_l12.to_csv(caminho_csv_aluno, index=False, encoding='utf-8-sig')

caminho_csv_trafego = r"C:\Users\Camilo_Bica\data_science\consultoria\acelerador_petrobras\dados\invest_trafego.csv"
df_leads_invest_trafego.to_csv(caminho_csv_trafego, index=False, encoding='utf-8-sig')

In [28]:
colunas_excluir = ["data", "email"]

for coluna in df_pesquisa_compradores_l12.columns:
    if coluna not in colunas_excluir:
        print(f"\nColuna: {coluna}")
        print(df_pesquisa_compradores_l12[coluna].value_counts(dropna=False))


Coluna: idade
idade
23-27         49
38-45         35
28-32         35
33-37         30
46-52         22
18-22         17
53 ou mais     6
Name: count, dtype: int64

Coluna: nível
nível
Técnica     163
Superior     31
Name: count, dtype: int64

Coluna: situação profissional
situação profissional
Funcionário de empresa privada                          98
Desempregado(a)                                         39
Autônomo                                                33
Empresário                                              12
Funcionário público                                     10
Já sou funcionário terceirizado do sistema Petrobras     2
Name: count, dtype: int64

Coluna: renda
renda
De 1.000 a 3.000         79
De 3.000 a 5.000         54
De 5.000 a 10.000        27
Estou desempregado(a)    21
De 0 a 1.000             10
Mais de 10.000            3
Name: count, dtype: int64

Coluna: escolaridade
escolaridade
Ensino médio completo         72
Ensino superior completo      61
Nível 